In [1]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [2]:
wiki_api_wrap = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=wiki_api_wrap)
wiki.name

'wikipedia'

In [4]:
arxiv_api_wrap = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=arxiv_api_wrap)
arxiv.name

'arxiv'

In [5]:
tools = [wiki, arxiv]

In [8]:
## Custom tools[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
os.environ['HF_TOKEN']= os.getenv('HF_TOKEN')
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

/opt/conda/envs/agent-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
loader=WebBaseLoader("https://paulgraham.com/best.html")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,embeddings)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x73cc721ff6b0>, search_kwargs={})

In [12]:
# Converting retriever into agentic tool

from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, 'Essay Search','Ask me about this essay...')
retriever_tool.name

'Essay Search'

In [13]:
tools = [wiki, arxiv, retriever_tool]

In [14]:
# Integrating tools with llms
from langchain_groq import ChatGroq
groq_api_key = os.getenv('GROQ_API_KEY')
llm = ChatGroq(groq_api_key=groq_api_key, model_name='Llama3-8b-8192')

In [16]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [17]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [18]:
# Agent Executer
from langchain.agents import AgentExecutor
agent_exectuor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_exectuor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [19]:
agent_exectuor.invoke({'input':'What is this article about ??'})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'What is this article about?'}`


Page: Whataboutism
Summary: Whataboutism or whataboutery (as in "what about ...?") is a pejorative for the strategy of responding to an accusation with a counter-accusation instead of a defense against the original accusation. 
From a logical and arg
Invoking: `Essay Search` with `{'query': 'What is this article about?'}`


The Best Essay

after decades of training, this is hard for new arrivals in the real
world to grasp. Helping them overcome such institutional lies will
work as long as the institutions remain broken. 
[12]Another recipe for timelessness is to write about things readers
already know, but in much more detail than can be transmitted
culturally. "Everyone knows," for example, that it can be rewarding
to have kids. But till you have them you don't know precisely what
forms that takes, and even then much of what you know you may never
have put into words.I've w

{'input': 'What is this article about ??',
 'output': 'I think I have the answer!\n\nThe article is about the concept of an essay and its characteristics. It explores what makes a good essay, including the importance of surprising readers with new information, being well-written, and being intellectually honest. The article also touches on the idea that essays can be timeless and that some topics are more suitable for essays than others.'}

In [20]:
agent_exectuor.invoke({'input':'What are some timeless essays ?? give me some focused on self transformation, life partner & relationship'})



> Entering new AgentExecutor chain...

Invoking: `Essay Search` with `{'query': 'timeless essays self transformation'}`


end to end by returning to a question you skipped earlier. Usually
it requires some sleight of hand to make the essay flow continuously
at this point, but not this time. This time I actually need an
example of the phenomenon. For example, we discovered earlier that
the best possible essay wouldn't usually be timeless in the way the
best painting would. This seems surprising enough to be
worth investigating further.There are two senses in which an essay can be timeless: to be about
a matter of permanent importance, and always to have the same effect
on readers. With art these two senses blend together. Art that
looked beautiful to the ancient Greeks still looks beautiful to us.
But with essays the two senses diverge, because essays
teach, and you can't teach people something they already know.
Natural selection is certainly a matter of permanent importance,
but an 

{'input': 'What are some timeless essays ?? give me some focused on self transformation, life partner & relationship',
 'output': 'The essays on self-transformation, life partner, and relationships are not explicitly mentioned in this output. However, there are some points that could be indirectly related to these topics:\n\n* The idea of "breadth of applicability" could be applied to self-transformation, as it involves finding ways to apply new knowledge and skills to one\'s life.\n* The concept of "novelty" could be relevant to finding a life partner, as it involves discovering new and unique individuals who can bring new experiences and perspectives to one\'s life.\n* The discussion of "timelessness" could be applied to relationships, as it involves finding ways to make a relationship last and remain meaningful over time.\n\nHowever, these connections are not explicit and would require further interpretation and analysis to be applied to the topics of self-transformation, life partn

In [21]:
agent_exectuor.invoke({'input':'Give me 10 greatest essays of 2024 ??'})



> Entering new AgentExecutor chain...

Invoking: `Essay Search` with `{'query': 'greatest essays 2024'}`


The Best Essay

March 2024Despite its title this isn't meant to be the best essay. My goal
here is to figure out what the best essay would be like.It would be well-written, but you can write well about any topic.
What made it special would be what it was about.Obviously some topics would be better than others. It probably
wouldn't be about this year's lipstick colors. But it wouldn't be
vaporous talk about elevated themes either. A good essay has to be
surprising. It has to tell people something they don't already know.The best essay would be on the most important topic you could tell
people something surprising about.That may sound obvious, but it has some unexpected consequences.
One is that science enters the picture like an elephant stepping
into a rowboat. For example, Darwin first described the idea of
natural selection in an essay written in 1844.
Talk about an
important 

{'input': 'Give me 10 greatest essays of 2024 ??',
 'output': 'I apologize, but it seems like the tool call id "call_vghd" yielded the same result as the previous tool call id "call_n0v7". It appears that the tool is stuck in a loop and is not providing any new or relevant information.\n\nAs a helpful assistant, I can try to provide a different response or explore other options to answer your question. However, I would like to clarify that the output from the tool seems to be a philosophical essay rather than a list of the greatest essays of 2024.\n\nIf you would like, I can try to provide a different response or explore other options to answer your question. Please let me know how I can assist you further.'}

In [22]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
ddg_search = DuckDuckGoSearchRun()
tools = [wiki, arxiv, retriever_tool, ddg_search]

In [27]:
agent_exectuor.invoke({'input':'Do you have memory of our past conversations ??'})



> Entering new AgentExecutor chain...
/tool-use>{"tool_calls": [{"id": "pending", "type": "function", "function": {"name": "wikipedia"}, "parameters": {"query": "memory"}}]}</tool-use>

> Finished chain.


{'input': 'Do you have memory of our past conversations ??',
 'output': '/tool-use>{"tool_calls": [{"id": "pending", "type": "function", "function": {"name": "wikipedia"}, "parameters": {"query": "memory"}}]}</tool-use>'}

In [30]:
tools = [ddg_search]

In [34]:
tools

[DuckDuckGoSearchRun(api_wrapper=DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='auto', source='text'))]

In [35]:
agent_exectuor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_exectuor


AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [36]:
# agent_exectuor.invoke({'input':'Do you like apples and oranges??'})

agent_exectuor.invoke({'input':'Give me 10 greatest essays of 2024 ??'})



> Entering new AgentExecutor chain...

Invoking: `Essay Search` with `{'query': 'greatest essays 2024'}`


Essay Search is not a valid tool, try one of [duckduckgo_search].

APIError: Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.